In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Sickle_Cell_Anemia/GSE46471'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Mechanisms of haptoglobin protection against hemoglobin peroxidation triggered endothelial damage"
!Series_summary	"Extracellular hemoglobin (Hb) has been recognized as a disease trigger in hemolytic conditions such as sickle cell disease, malaria and blood transfusion. In vivo, many of the adverse effects of free Hb can be attenuated by the Hb scavenger acute phase protein haptoglobin (Hp). The primary physiologic disturbances that can be caused be free Hb are found within the cardiovascular system and Hb triggered oxidative toxicity towards the endothelium has been promoted as a potential mechanism. The molecular mechanisms of this toxicity as well as of the protective activities of Hp are not yet clear. Within this study we systematically investigated the structural, biochemical and cell biologic nature of Hb toxicity in an endothelial cell system under peroxidative stress."
!Series_summary	"We identified two principal mechanisms of oxidative H

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

is_gene_available = True

# Determine the keys for available variables
sample_characteristics = {
    0: ['cell type: HPAEC', 'cell type: HUVEC'],
    1: ['treatment: no treatment'],
    2: ['time: 0h']
}

# Analyze and determine availability and keys
trait_row = None  # No explicit mention of Sickle_Cell_Anemia
age_row = None   # No explicit mention of age
gender_row = None  # No explicit mention of gender

# Define conversion functions
def convert_trait(value):
    return None  # No conversion since trait data is not available

def convert_age(value):
    return None  # No conversion since age data is not available

def convert_gender(value):
    return None  # No conversion since gender data is not available

# Save cohort information
save_cohort_info('GSE46471', './preprocessed/Sickle_Cell_Anemia/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Sickle_Cell_Anemia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Sickle_Cell_Anemia/trait_data/GSE46471.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
